# STK4030 Final project

## Setup

In [46]:
set.seed(4030)
library(caret)
library(mboost)

## Helper functions

In [4]:
AllColumnsExcept = function(data, col.names) {
    columns = !(names(data) %in% col.names)
    return(data[, columns])
}

In [5]:
SortData = function(data) {
    data.as.df = as.data.frame(data)
    sorted.data = data.as.df[with(data.as.df, order(-abs(data.as.df[, 1]))), , drop = FALSE]
    return(sorted.data)
}

## Problem 1

### Load data

In [6]:
load("bostonhousing.rdata")

training.data = data[data$train == TRUE, ]
training.data = AllColumnsExcept(training.data, 'train')

test.data = data[data$train == FALSE, ]
test.data = AllColumnsExcept(test.data, 'train')

### 1.1 Linear regression

*Estimate a linear Gaussian regression model including all 14 independent variables by (ordinary) least squares (OLS) on the training set.*

In [7]:
lgr.model = lm(y ~ ., training.data)

*Report the estimated coefficients.*

In [8]:
summary(lgr.model)
lgr.model$coefficients


Call:
lm(formula = y ~ ., data = training.data)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.63205 -0.10022 -0.00941  0.08597  0.71839 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.4269062  0.2592184  17.078  < 2e-16 ***
crim        -0.0092742  0.0016103  -5.759 2.60e-08 ***
zn           0.0014557  0.0007420   1.962 0.050929 .  
indus        0.0035924  0.0032292   1.112 0.267057    
chas         0.1226629  0.0434888   2.821 0.005198 ** 
nox         -0.6707267  0.1983857  -3.381 0.000844 ***
rm           0.0699879  0.0212350   3.296 0.001131 ** 
age          0.0001011  0.0007072   0.143 0.886416    
dis         -0.0619333  0.0103753  -5.969 8.59e-09 ***
rad          0.0144673  0.0034732   4.165 4.35e-05 ***
tax         -0.0005983  0.0001945  -3.077 0.002339 ** 
ptratio     -0.0355679  0.0071460  -4.977 1.24e-06 ***
bk          -0.3992916  0.1027225  -3.887 0.000132 ***
lstat       -0.0363930  0.0026695 -13.633  < 2e-16 ***
part   

(Intercept)          crim            zn         indus          chas 
 4.4269062382 -0.0092742305  0.0014557439  0.0035923854  0.1226629404 
          nox            rm           age           dis           rad 
-0.6707267346  0.0699878708  0.0001011223 -0.0619332911  0.0144673164 
          tax       ptratio            bk         lstat          part 
-0.0005982967 -0.0355678980 -0.3992916432 -0.0363929598 -0.0089272055

*Which covariates have the strongest association with y? In particular, the study focused on the effect of air pollution, measured through the concentrations of nitrogen oxide pollutants (nox) and particulate (part).*

#### Correlations

In [9]:
correlations = cor(training.data, training.data['y'], method = "pearson")
SortData(correlations)

,y
y,1.00000000
lstat,-0.84057651
rm,0.62856668
tax,-0.56875320
indus,-0.55433246
nox,-0.52532741
crim,-0.52429359
ptratio,-0.50639657
rad,-0.49478292
age,-0.48137089


#### Coefficients on a standardized regression model

In [10]:
scaled.training.data = lapply(training.data, scale)
scaled.model = lm(y ~ ., scaled.training.data)o

,data
lstat,-6.245181e-01
rad,3.162886e-01
dis,-3.095360e-01
tax,-2.538477e-01
crim,-2.029852e-01
nox,-1.977650e-01
ptratio,-1.907376e-01
rm,1.197933e-01
bk,-1.135118e-01
zn,8.091898e-02


In [ ]:
SortData(scaled.model$coefficients)

#### Incremental/partial R2

Gain in R2 when adding variable as the last one
TODO

*Do they have any effect on the house price? If yes, which kind of effect?*

- **part** has a (statistically insignificant) negative effect on housing prices
- **nox** has a (statistically significant) negative effect on housing prices

### 1.2 Evaluation

*The model above can be also used to predict the price for the other tracts (test set).*

*Compute the prediction error on the test data.*

In [12]:
RMSE = function(model, data) {
  predictions = predict(model, data)
  prediction.error = sqrt(mean((predictions - data$y)^2))
  return(prediction.error)
}

In [15]:
RMSE(lgr.model, test.data)

[1] 0.2147611

*Moreover, derive two reduced models by applying a backward elimination procedure with AIC and α = 0.05 as stopping criteria, respectively. For both models, report the estimated coefficients and the prediction error estimated on the test data. Comment the results.*

#### Backard elimination with AIC as stopping criteria

In [14]:
aic.elimination.model = lm(y ~ ., training.data)
aic.elimination.model = step(aic.elimination.model, direction = "backward")

Start:  AIC=-882.01
y ~ crim + zn + indus + chas + nox + rm + age + dis + rad + tax + 
    ptratio + bk + lstat + part

          Df Sum of Sq     RSS     AIC
- age      1    0.0006  6.8805 -883.99
- part     1    0.0252  6.9051 -883.09
- indus    1    0.0358  6.9156 -882.70
<none>                  6.8799 -882.01
- zn       1    0.1113  6.9911 -879.95
- chas     1    0.2300  7.1098 -875.69
- tax      1    0.2736  7.1535 -874.14
- rm       1    0.3140  7.1939 -872.72
- nox      1    0.3304  7.2103 -872.14
- bk       1    0.4368  7.3166 -868.44
- rad      1    0.5016  7.3814 -866.21
- ptratio  1    0.7161  7.5960 -858.96
- crim     1    0.9588  7.8387 -851.00
- dis      1    1.0300  7.9099 -848.71
- lstat    1    5.3725 12.2524 -738.00

Step:  AIC=-883.99
y ~ crim + zn + indus + chas + nox + rm + dis + rad + tax + ptratio + 
    bk + lstat + part

          Df Sum of Sq     RSS     AIC
- part     1    0.0263  6.9068 -885.02
- indus    1    0.0357  6.9162 -884.68
<none>                  6

In [15]:
aic.elimination.model$coefficients

(Intercept)          crim            zn          chas           nox 
 4.3583175609 -0.0093233318  0.0013472107  0.1338892502 -0.5878773979 
           rm           dis           rad           tax       ptratio 
 0.0681740354 -0.0628805947  0.0131911759 -0.0004905084 -0.0340597069 
           bk         lstat 
-0.4029111646 -0.0361469832

In [16]:
RMSE(aic.elimination.model, test.data)

[1] 0.2143025

Removes age, part, indus

#### Backward elimination with α = 0.05 as stopping criteria

Comment: Use p-value as both stopping and selection criteria

In [17]:
PValues = function(model) {
  return(summary(model)$coefficients[-1, 4])
}

In [18]:
alpha.elimination.model = NULL
alpha.elimination.data = training.data

repeat {
  alpha.elimination.model = lm(y ~ ., alpha.elimination.data)

  if (all(PValues(alpha.elimination.model) <= 0.05)) {
    break;
  }
  
  p.values = PValues(alpha.elimination.model)
  highest.p.value = p.values[which.max(p.values)]
  predictor.with.highest.p.value = names(highest.p.value)
  alpha.elimination.data = AllColumnsExcept(alpha.elimination.data, predictor.with.highest.p.value) 
}

In [19]:
alpha.elimination.model$coefficients

(Intercept)          crim          chas           nox            rm 
 4.4075094989 -0.0090857605  0.1300907298 -0.6277073471  0.0727657497 
          dis           rad           tax       ptratio            bk 
-0.0537378737  0.0132326503 -0.0004450344 -0.0392956598 -0.4027808969 
        lstat 
-0.0360381787

In [20]:
RMSE(alpha.elimination.model, test.data)

[1] 0.2144203

Discuss: P-value removes one more variable (zn) than AIC.

Discuss: Inflated p-value (increase risk of type 1 error)

### 1.3 Principal Component Regression

*Estimate a principal component regression model, selecting the number of components by 10-fold cross-validation.*

Discuss: Standardizion

Consider: Use oneSE rule

In [35]:
PCRModel = function(data, method, number) {
    set.seed(4030)
    model = train(
        y ~ .,
        data,
        method = "pcr",
        # Include 1 to 14 principal components
        tuneGrid = expand.grid(ncomp = 1:14),
        # During CV, pre-processing parameters are determined from 9 of the 10 folds, then applied to the 10th
        preProcess = c("scale"),
        trControl = trainControl(
            method = method,
            number = number
        ))
    model$num.of.components = model$bestTune$ncomp
    return(model)
}

In [36]:
pcr.cv.model = PCRModel(training.data, 'CV', 10)
pcr.cv.model$results

ncomp,RMSE,Rsquared,MAE,RMSESD,RsquaredSD,MAESD
1,0.2763718,0.5378163,0.2108739,0.07239596,0.17255558,0.05506233
2,0.2714324,0.5610061,0.2079697,0.06343427,0.15437409,0.04227620
3,0.2482814,0.6376104,0.1795553,0.05798842,0.13194245,0.03580606
4,0.2387057,0.6630701,0.1712795,0.05183052,0.11997343,0.03193207
5,0.2175733,0.7207076,0.1522210,0.05443111,0.12270545,0.03135261
6,0.2072923,0.7410145,0.1481009,0.06556232,0.13770523,0.03680555
7,0.2087002,0.7368135,0.1474513,0.06663998,0.13790149,0.03622257
8,0.2014379,0.7647016,0.1411029,0.06494232,0.12977845,0.03612086
9,0.1908346,0.7835779,0.1374645,0.06145838,0.12001075,0.03609148
10,0.1935024,0.7821185,0.1407182,0.05522173,0.10913354,0.03155260


In [37]:
RMSE(pcr.cv.model, test.data)

[1] 0.2147611

*How many components have been selected?*

In [38]:
pcr.cv.model$num.of.components

[1] 14

*What does it mean?*

TODO

Discuss: All variables contribute with some variation

### 1.4 PCR using .632 bootstrap

*Repeat the procedure to choose the number of components by using the .632 bootstrap procedure.*

In [39]:
pcr.bootstrap.model = PCRModel(training.data, 'boot632', 100)
pcr.bootstrap.model$results

ncomp,RMSE,Rsquared,MAE,RMSESD,RsquaredSD,MAESD,RMSEApparent,RsquaredApparent,MAEApparent
1,0.2863948,0.5009818,0.2126434,0.02515808,0.07118448,0.01657741,0.2846565,0.5051366,0.2109639
2,0.2751841,0.5388360,0.2061874,0.02679436,0.07424027,0.01775282,0.2715134,0.5497791,0.2036324
3,0.2555808,0.6026588,0.1835885,0.02884978,0.07875032,0.02010413,0.2484271,0.6230872,0.1764341
4,0.2443959,0.6358841,0.1741528,0.02808502,0.07712200,0.01842157,0.2383149,0.6531471,0.1683601
5,0.2284280,0.6803315,0.1587133,0.03157019,0.08603907,0.02103806,0.2131813,0.7224500,0.1438064
6,0.2223465,0.6975219,0.1538384,0.03070494,0.08177574,0.01925407,0.2124619,0.7243200,0.1450717
7,0.2177125,0.7100870,0.1484483,0.02908912,0.07611478,0.01614699,0.2113829,0.7271130,0.1417871
8,0.2085486,0.7347672,0.1418587,0.02465259,0.06417257,0.01328933,0.2001549,0.7553330,0.1346697
9,0.1984373,0.7591913,0.1380014,0.02386845,0.05729907,0.01274869,0.1887829,0.7823452,0.1307096
10,0.1967646,0.7633898,0.1381763,0.02347863,0.05791045,0.01226595,0.1864402,0.7877137,0.1302930


In [43]:
RMSE(pcr.bootstrap.model, test.data)

[1] 0.2147611

*Does the number of selected components change?*

In [41]:
pcr.bootstrap.model$num.of.components

[1] 14

*Report the estimate of the prediction error for each possible number of components.*

### Regularized regression

### 1.5 Ridge regression

*Estimate the regression model by ridge regression, where the optimal tuning parameter λ is chosen by 10-fold cross-validation.*

**TODO**: What are reasonable lambda values?

In [72]:
ridge.model = train(
  y ~ .,
  data = training.data,
  method = "glmnet",
  tuneGrid = expand.grid(
    # alpha = 0: LM2
    alpha = 0,
    lambda = seq(0.00, 1.00, 0.01)
  ),
  trControl = trainControl(
    method = 'cv',
    number = 10
  )
)

*Report the estimated coefficients, the obtained value of lambda and the prediction error computed on the test data.*

In [73]:
as.matrix(coef(ridge.model$finalModel, ridge.model$bestTune$lambda))

,1
(Intercept),3.8771124229
crim,-0.0080244317
zn,0.0008041282
indus,-0.0002841339
chas,0.1359930995
nox,-0.3888605767
rm,0.0965529950
age,-0.0004318776
dis,-0.0441903838
rad,0.0061303460


In [76]:
ridge.model$bestTune$lambda

[1] 0.03

In [77]:
RMSE(ridge.model, test.data)

[1] 0.2095294

### 1.6 Lasso regression

*Repeat the same procedure by using lasso and component-wise L2Boost. Use 10-fold cross-validation to find the optimal value for λ (lasso) and mstop (L2Boost), while set the boosting step size ν equal to 0.1.*

##### Lasso

In [94]:
set.seed(4030)
lasso.model = train(
  y ~ .,
  data = training.data,
  method = "glmnet",
  tuneGrid = expand.grid(
    # alpha = 1: L1 penalty
    alpha = 1,
    lambda = 10^seq(-5, 5, by = .1)
  ),
  trControl = trainControl(
    method = 'cv',
    number = 10
  )
)

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

In [95]:
lasso.model$bestTune['lambda']

,lambda
25,0.002511886


In [96]:
as.matrix(coef(lasso.model$finalModel, lasso.model$bestTune$lambda))

,1
(Intercept),4.1879395859
crim,-0.0083739274
zn,0.0009906054
indus,0.0000000000
chas,0.1300856832
nox,-0.4781236371
rm,0.0725376014
age,0.0000000000
dis,-0.0537543990
rad,0.0087722543


In [97]:
RMSE(lasso.model, test.data)

[1] 0.2146437

#### L2Boost

In [84]:
set.seed(4030)
l2boost.models = glmboost(
    y ~ .,
    data = training.data,
    family = Gaussian(),
    # TODO: Why center data?
    center = TRUE,
    control = boost_control(
        mstop = 5000,
        # Step size
        nu = 0.1
    )
)

l2boost.cv = cvrisk(
    l2boost.models,
    folds = cv(
        model.weights(l2boost.model2),
        type = 'kfold',
        B = 10
    )
)

In [85]:
l2boost.mstop = mstop(l2boost.cv)
l2boost.mstop

[1] 1543

In [86]:
l2boost.model = l2boost.models[l2boost.mstop]
coef(l2boost.model)

(Intercept)          crim            zn         indus          chas 
 1.2698893996 -0.0088666291  0.0012742634  0.0023128607  0.1313974140 
          nox            rm           dis           rad           tax 
-0.5818622760  0.0730271467 -0.0590176414  0.0122480653 -0.0004872262 
      ptratio            bk         lstat          part 
-0.0337409496 -0.3881370861 -0.0367312690 -0.0078850454 
attr(,"offset")
[1] 3.042633

In [83]:
RMSE(l2boost.model, test.data)

[1] 0.2152984

### 1.7 Variable transformations

*It has been argued that the predictors rm and dis do not have a linear effect on the outcome. Substitute the former with its cube and the latter with its inverse (dis-1) in the first model (OLS) and refit the model.*

In [87]:
TransformData = function(data) {
  transformed.data = data
  transformed.data$rm.cubed = data$rm^3
  transformed.data$dis.inverse = data$dis^(-1)
  transformed.data = AllColumnsExcept(transformed.data, c('rm', 'dis'))
}

In [88]:
transformed.training.data = TransformData(training.data)
transformed.test.data = TransformData(test.data)

In [89]:
transformed.model = lm(y ~ ., transformed.training.data)

*Report the estimated coefficients.*

In [90]:
transformed.model$coefficients

(Intercept)          crim            zn         indus          chas 
 4.1498013458 -0.0129132713 -0.0003568615  0.0019736474  0.1306422094 
          nox           age           rad           tax       ptratio 
-0.8701347958 -0.0008381449  0.0144924858 -0.0006314347 -0.0284772490 
           bk         lstat          part      rm.cubed   dis.inverse 
-0.3350774831 -0.0357424965 -0.0077779790  0.0009468631  0.9615618907

*Compute the prediction error on the test set and compare the result with that obtained at point 1.*

In [91]:
RMSE(transformed.model, transformed.test.data)

[1] 0.2103309

In [93]:
lgr.prediction.error

ERROR: Error in eval(expr, envir, enclos): object 'lgr.prediction.error' not found


The fit improved!

TODO: Comment/test significance of difference

F-test

Plots